<a href="https://colab.research.google.com/github/tnwlvos/machinelearning2/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:

import os, random, shutil

# 기본 경로 설정
base = "/content/datasets/coco"
train_img_dir = os.path.join(base, "images/train2017")
train_label_dir = os.path.join(base, "labels/train2017")

# 새로운 샘플 데이터셋 경로
sample_dir = "/content/datasets/coco_300"
os.makedirs(os.path.join(sample_dir, "images/train2017"), exist_ok=True)
os.makedirs(os.path.join(sample_dir, "labels/train2017"), exist_ok=True)

# 클래스별 최대 이미지 수
max_per_class = 300

# 클래스별 카운터
class_counter = {i: 0 for i in range(80)}

# 라벨 파일 순회
label_files = [f for f in os.listdir(train_label_dir) if f.endswith(".txt")]
random.shuffle(label_files)

for label_file in label_files:
    label_path = os.path.join(train_label_dir, label_file)
    with open(label_path, "r") as f:
        lines = f.readlines()

    # 파일 내 클래스 목록
    classes_in_file = {int(line.split()[0]) for line in lines}

    # 만약 이 이미지의 클래스들이 아직 300장 미만이면 복사
    if any(class_counter[c] < max_per_class for c in classes_in_file):
        # 이미지 & 라벨 복사
        img_name = label_file.replace(".txt", ".jpg")
        src_img = os.path.join(train_img_dir, img_name)
        dst_img = os.path.join(sample_dir, "images/train2017", img_name)
        dst_label = os.path.join(sample_dir, "labels/train2017", label_file)

        if os.path.exists(src_img):
            shutil.copy(src_img, dst_img)
            shutil.copy(label_path, dst_label)

            # 카운터 증가
            for c in classes_in_file:
                class_counter[c] += 1

    # 80개 클래스 모두 300장 채워졌으면 종료
    if all(v >= max_per_class for v in class_counter.values()):
        break

print("✅ 클래스별 최대 300장씩 샘플링 완료!")
print(f"총 복사된 이미지 수: {len(os.listdir(os.path.join(sample_dir, 'images/train2017')))}")

import yaml

data_sample = {
    'train': '/content/datasets/coco_300/images/train2017',
    'val': '/content/datasets/coco/images/val2017',  # 기존 val 그대로 사용
    'nc': 80,
    'names': [
        'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck',
        'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
        'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
        'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
        'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
        'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse',
        'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
        'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
}

with open('coco_300.yaml', 'w') as f:
    yaml.dump(data_sample, f)

print("✅ coco_300.yaml 생성 완료")
%cd /content/yolov5
!python train.py --data /content/datasets/coco_300.yaml --weights yolov5.pt --epochs 2 --batch 32 --img 640 --name coco_300

%cd /content/yolov5
!python detect.py --weights runs/train/exp/weights/best.pt --source data/images --conf 0.25

import glob
from IPython.display import Image, display

# 탐지된 이미지 파일 중 일부 표시
result_images = glob.glob('runs/detect/exp2/*.jpg')

for img_path in result_images[:3]:  # 상위 3개 예시만
    display(Image(filename=img_path, width=600))



#coco128
!rm -rf /content/yolov5/runs
!rm -rf /content/datasets/coco*/**/*.cache
!rm -rf /content/datasets/coco*/**/*.pt

%cd /content/yolov5
!python train.py --data data/coco128.yaml --weights yolov5s.pt --epochs 10 --batch 16 --img 640 --name coco128_clean

!python detect.py --weights runs/train/coco128_clean/weights/best.pt --source data/images --conf 0.25
import glob
from IPython.display import Image, display

result_images = glob.glob('runs/detect/exp/*.jpg')
for img_path in result_images[:3]:
    display(Image(filename=img_path, width=600))




# ============================================
# 1️⃣ 환경 준비 및 캐시 정리
# ============================================
%cd /content

# yolov5 디렉토리 없으면 다시 클론
!rm -rf yolov5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt -q

# 이전 학습 및 캐시 데이터 삭제
!rm -rf /content/yolov5/runs
!find /content/datasets -name "*.cache" -delete


# ============================================
# 2️⃣ 기존 하이퍼파라미터 확인 (기본값)
# ============================================
import yaml

default_hyp_path = "data/hyps/hyp.scratch-low.yaml"

with open(default_hyp_path, 'r') as f:
    default_hyp = yaml.safe_load(f)

print("📘 [기존 YOLOv5 기본 하이퍼파라미터]\n")
for k, v in default_hyp.items():
    print(f"{k:<20}: {v}")

!pkill -f train.py

# 2️⃣ 캐시/진화 로그 싹 정리
!rm -rf /content/yolov5/runs/hyp_opt
!rm -rf /content/datasets/coco128/*.cache

# 3️⃣ 다시 YOLO 디렉토리 진입
%cd /content/yolov5

# ============================================
# 3️⃣ 하이퍼파라미터 자동 진화 (evolve)
# ============================================
!python train.py --data data/coco128.yaml \
                 --weights yolov5s.pt \
                 --epochs 10 \
                 --batch 16 \
                 --img 640 \
                 --evolve 1 \
                 --project runs/hyp_opt \
                 --name evolve_test


# ============================================
# 4️⃣ 최적화된 하이퍼파라미터 결과 확인
# ============================================
import os

best_hyp_path = "runs/hyp_opt/evolve_test/hyp_evolve.yaml"

if os.path.exists(best_hyp_path):
    with open(best_hyp_path, 'r') as f:
        evolved_hyp = yaml.safe_load(f)

    print("\n\n✅ [최적화된 하이퍼파라미터 결과]\n")
    for k, v in evolved_hyp.items():
        print(f"{k:<20}: {v}")
else:
    print("⚠️ 아직 evolve 결과가 생성되지 않았습니다.")


# ============================================
# 5️⃣ 기존 vs 최적화된 파라미터 비교
# ============================================
print("\n\n🔍 [기존 vs 최적화된 하이퍼파라미터 비교]\n")
for k in default_hyp.keys():
    old = default_hyp[k]
    new = evolved_hyp.get(k, None)
    if new is not None:
        changed = "⬆️ 증가" if new > old else ("⬇️ 감소" if new < old else "➖ 동일")
        print(f"{k:<20}: 기존={old:<10} | 변경={new:<10} | 변화={changed}")

!python train.py --data coco128.yaml --epochs 10 --batch-size 16 --weights yolov5s.pt --img 640 --hyp /content/yolov5/runs/hyp_opt/evolve_test/hyp_evolve.yaml



!python train.py --data coco128.yaml --epochs 10 --batch-size 16 --weights yolov5n.pt --name yolov5n_exp

!python train.py --data coco128.yaml --epochs 10 --batch-size 16 --weights yolov5m.pt --name yolov5m1_exp

!python train.py --data coco128.yaml --epochs 10 --batch-size 16 --weights yolov5l.pt --name yolov5l_exp

!python train.py --data coco128.yaml --epochs 10 --batch-size 16 --weights yolov5x.pt --name yolov5x_exp






wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-11-04 13:03:22.433014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762261402.454888  104707 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762261402.461276  104707 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762261402.478046  104707 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762261402.478077  104707 computation_placer.cc:177] computation placer already registere